# Homework 1 - Linear Regression

## Dataset
The dataset you will be using is about Life expectancy of different countries. We will explore how immunization factors, mortality factors, economic factors, social factors and other health related factors affect Life expectancy of a country.

There are two data files: "LifeExpectancy_training_modified.csv" and "LifeExpectancy_test_modified.csv"<br/>
Both files have the following fields, except Life_expectancy which is not available in "LifeExpectancy_test_modified.csv"

Features :
- Year : from 2002 to 2015
- Status : Developed or Developing status
- Adult_Mortality : Adult Mortality Rates of both sexes (probability of dying between 15 and 60 years per 1000 population)
- Alcohol : Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)
- percentage_expenditure : Expenditure on health as a percentage of Gross Domestic Product per capita(%)
- BMI: Average Body Mass Index of entire population
- Total_expenditure: General government expenditure on health as a percentage of total government expenditure (%)
- Diphtheria: Diphtheria tetanus toxoid and pertussis (DTP3) immunization coverage among 1-year-olds (%)
- HIV_AIDS: Deaths per 1000 live births HIV/AIDS (0-4 years)
- GDP: Gross Domestic Product per capita (in USD)
- Population
- Income_composition_of_resources: Human Development Index in terms of income composition of resources (index ranging from 0 to 1)
- Schooling: Number of years of Schooling(years)
- Health_Index: Health index

Target:
- Life_expectancy: Life Expectancy in age


Training dataset, "LifeExpectancy_training_modified.csv", contains 1064 rows and 15 columns. This is the training set containing both of the features and the target.<br/>
Test dataset, "LifeExpectancy_test_modified.csv", contains 458 rows and 14 columns. This is the test set which only contains the features.<br/>

Your goal is to predict Life expectancy based on the features.

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from statistics import mean
from sklearn import datasets
import seaborn as sns


Load the training data "LifeExpectancy_training_modified.csv" in Colab and View the first 5 lines

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Load the training data
import io
df_train = pd.read_csv(io.BytesIO(uploaded['LifeExpectancy_training_modified.csv']))

In [ ]:
# Show the first 5 lines
### WRITE CODE ###
df_train.head()

## Data Exploration
We can plot a histogram of the dataframe for the features except "Status" to understand their distributions. <br/>

In [ ]:
### WRITE CODE TO OBTAIN AND DISPLAY HISTOGRAMS ###

import matplotlib.pyplot as plt

# Make a copy of the dataframe with the features except "Status"
features_without_status = df_train.drop(columns=['Status'])

# Plot a histogram for each of the rest of the features

print("\nDistributions:")
for column in features_without_status.columns:
    plt.figure(figsize=(8, 4))
    if features_without_status[column].dtype == 'object':  # Categorical data
        sns.countplot(x=features_without_status[column])
    else:  # Numerical data
        sns.histplot(features_without_status[column], kde=True)
    plt.title(f'Distribution of {column}')
    plt.show()

##### Q1. What can you infer from the histograms? <br/>
From the histograms, we can infer that the features Total_expenditure and Schooling have an approximately normal distribution, the features Adult_Mortality, ALcohol, percentage_expenditure, HIV_AIDS, GDP, and Population are right-skewed, and the features Diphtheria, Income_composition_of_resources, and Life_expectancy are left-skewed, while the rest of the features do not display a clear distribution.
The features with a highly skewed distribution might have a nonlinear relationship with another variable.

Compute the correlation matrix to get an understanding of the correlation between life_expectancy and the other features.<br/>

In [ ]:
### WRITE CODE TO OBTAIN CORRELATION MATRIX ###

# Compute the correlation matrix
corr_matrix = df_train.corr()
pd.DataFrame(corr_matrix)

In [ ]:
# Correlation between life_expectancy and the other features
corr_matrix = corr_matrix['Life_expectancy'].drop('Life_expectancy')
pd.DataFrame(corr_matrix)

##### Answer the following questions:<br/>

##### Q2. Why is the diagonal made up of 1's in the correlation matrix?<br/>
Because the the number making up the diagonal of a correlation matrix is the correlation between a variable with itself, which should always be 1.

##### Q3. Why is the matrix symmetric along diagonal?<br/>
Because the correlation between a first and a second variable always equals to that between the second and the first.

##### Q4. Looking at the correlation matrix, if you have to choose one predictor for a simple linear regression model with Life_expectancy as the outcome, which one would you choose and why? <br/>
I would choose Income_composition_of_resources because it has the highest correlation with Life_expectancy, which is 0.756515, implying a strong linear relationship between Income_composition_of_resources and Life_expectancy.

##### Q4.1. Is there any variable that does not make sense to you and why? <br/>
The BMI variable does not make sense to me because the it has a relatively strong positive relationship with Life_expectancy with a correlation of 0.557677. This implies that the higher the BMI is, the higher the Life_expectancy, which is not the case. BMI should be within a certain range to improve life expectancy.

### Standardization of features

Feature standardization makes the values of each feature in the data have zero-mean and unit-variance. This method is widely used for normalization in many machine learning algorithms. The general method of calculation is to determine the distribution mean and standard deviation for each feature. Next we subtract the mean from each feature. Then we divide the values of each feature by its standard deviation.

$x'$ = ($x$ - $\bar{x}$)/$\sigma$

where $x$ is the original feature vector,
$\bar{x}$ is the mean of the feature vector and
$\sigma$ is its standard deviation.

This is also called Z-score Normalization.

Perform Z-score Normalization on the features (except "Year" and "Status") in both training and test set.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Load the test set "LifeExpectancy_test_modified.csv"
import io
df_test = pd.read_csv(io.BytesIO(uploaded['LifeExpectancy_test_modified.csv']))

In [ ]:
### WRITE CODE TO PERFORM Z-score Normalization ###

from sklearn.preprocessing import StandardScaler

# Selecting numerical data
numerical_data = df_train.drop(columns=['Year', 'Status', 'Life_expectancy'])

# Applying StandardScaler
scaler = StandardScaler()
numerical_scaled = scaler.fit_transform(numerical_data)

# Convert to DataFrame
numerical_scaled_df_train = pd.DataFrame(numerical_scaled, columns=numerical_data.columns)

# Display the first few rows of the scaled dataframe
numerical_scaled_df_train.head()



In [ ]:
# Selecting numerical data
numerical_data = df_test.drop(columns=['Year', 'Status'])

# Applying StandardScaler
scaler = StandardScaler()
numerical_scaled = scaler.fit_transform(numerical_data)

# Convert to DataFrame
numerical_scaled_df_test = pd.DataFrame(numerical_scaled, columns=numerical_data.columns)

# Display the first few rows of the scaled dataframe
numerical_scaled_df_test.head()

##### Q5. What are the advantages and disadvantages of using Z-score Normalization?<br/>
Advantages include: It makes the data easier to interpret on a standardized scale; It does not change the distribution of the original data.
Disadvantages include: Outliers might affect the normalization; It might not be applied to certain datasets, such as those with non-normal or highly-skewed distribution.

##### Q6. In this dataset, do you need to use the Z-score Normalization? Explain.<br/>
Yes. All the features are on different scales and units. Z-score Normalization makes it easier to compare the distributions between features.

### One-Hot Encoding

"Year" and "Status" can only take discrete values. We need to perform one-hot encoding on discrete values for it to be processed in the model. One hot encoding is a process by which categorical variables are converted into a form that could be provided to ML algorithms to do a better job in prediction.
Perform one-hot encoding on "Year" and "Status" and print the shape of your encoded array

In [ ]:
from sklearn.preprocessing import OneHotEncoder
### WRITE CODE TO PERFORM ONE-HOT CODING ON "Year" AND "Status" ###
one_hot_training = df_train[['Year', 'Status']]

encoder = OneHotEncoder(sparse=False)
categorical_encoded_training = encoder.fit_transform(one_hot_training)

categorical_encoded_df_training = pd.DataFrame(categorical_encoded_training, columns=encoder.get_feature_names_out(one_hot_training.columns))

one_hot_test = df_test[['Year', 'Status']]

encoder = OneHotEncoder(sparse=False)
categorical_encoded_test = encoder.fit_transform(one_hot_test)

categorical_encoded_df_test = pd.DataFrame(categorical_encoded_test, columns=encoder.get_feature_names_out(one_hot_test.columns))


categorical_encoded_df_training.head()


In [ ]:
categorical_encoded_df_test.head()

In [ ]:
print(categorical_encoded_df_training.shape)
print(categorical_encoded_df_test.shape)

Q7. What are the other types of encodings and why did we use One-hot encoding for "Year" and "Status"?

Ans- A few other types of encodings are:

1. Label Encoding: Here we assign an integer to category. It is usually used for ordinal categorical variables.

2. Binary Encoding: Here, first the categories are converted into integers, which are then converted into binary code. It is more efficient that one-hot encoding if the number of categories are high.

3. Frequency Encoding: Here, each category is replaced with its frequency in the dataset.

4. Mean Encoding: Here, each categorical value is replaced with the mean of the target variable for that category.


One-hot encoding is the perfect choice for 'Status' because it prevents the model to establish a numerical relationship between the categories.

Here 'Year' is encoded using One-hot encoding because it acts like nominal variable and not a numeric variable. Moreover, since there are only 14 distinct years in the dataset, the issue of high dimensionality can be managed.

## Multiple Linear Regression

In the big data era, it is highly unlikely that we are interested in the effect of a single variable on another. To simultaneously account for the effects of multiple variables, we use multiple regression (which accounts for the covariances between predictors).

While the algorithmic solution to multiple regression exists, it is easier to conceptualize in terms of linear algebra. The optimal $\hat{\beta}$ vector that minimizes the residual sum of squares is:

$\hat{\beta} = (X^TX)^{-1}X^Ty $


Perform multiple linear regression on the training dataset, where the outcome is "Life_expectancy".

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Combine the df
df_train_encoded = pd.concat([numerical_scaled_df_train, categorical_encoded_df_training, df_train['Life_expectancy']], axis=1, join='inner')
df_test_encoded = pd.concat([numerical_scaled_df_test, categorical_encoded_df_test], axis=1, join='inner')

In [ ]:
### Bulding and fitting the Multiple Linear Regression model###
X = df_train_encoded.drop('Life_expectancy', axis=1)
y = df_train_encoded['Life_expectancy']

model = LinearRegression()

model.fit(X, y)

In [ ]:
### Evaluate the Linear Regression model by computing MSE on the training set###
from sklearn.metrics import mean_squared_error

y_train_pred = model.predict(X)
mse_train = mean_squared_error(y, y_train_pred)

print("Mean Squared Error on Training Set:", mse_train)

Q8. Print the value of coefficients and also the corresponding variable names for the coefficients.

In [ ]:
coefficients = model.coef_

# Get the feature names
feature_names = X.columns

coefficients_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

coefficients_df['Coefficient'] = coefficients_df['Coefficient'].apply(lambda x: f'{x:.4f}')

coefficients_df

In [ ]:
#Correlation HeatMap
corr_matrix = df_train_encoded.corr()

plt.figure(figsize=(20, 18))  # Size of the figure
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

Q9. Is there a problem of multicolinearity? Explain what you can do

Ans- In the above correlation heatmap, leaving aside the diagonal, we can see that there is a high correlation between a few features. Therefore, there is a problem of multicolinearity.


Multicollinearity can be dealt using the following methods:

1. Remove the variables with high collinearity.
2. Combine variables using feature engineering
3. Use Principal Component Analysis
4. Regularization like ridge and lasso can be applied

### Goodness of fit

A model can always make predictions. But it is important to determine how good the model is.
How do we know that our model captures the data well? When evaluating model fit, a good metric is $R^2$, which corresponds to the amount of variance explained by the model. The formula for $R^2$ is the following:

$R^2$ = $1 - \dfrac{RSS}{TSS}$<br/>
where:<br/>
$RSS = \Sigma(y - \hat{y})^2$<br/>
$TSS = \Sigma(y - \bar{y})^2$<br/>

$R^2$ is also one metric for comparing models against each other. It is intuitive to say that the model that explains more variation in the data is a better fit than one that explains less variation.

Fill in the code for calculation of R2 score

In [ ]:
from sklearn.metrics import r2_score

$R^2$ for model with "Schooling" as predictor and "Life_expectancy" as outcome

In [ ]:
### WRITE CODE ###
X = df_train_encoded[['Schooling']]
y = df_train_encoded['Life_expectancy']

model = LinearRegression()
model.fit(X,y)

y_pred = model.predict(X)
r2 = r2_score(y,y_pred)
# Print R2 score
print('R^2 for model with "Schooling" as predictor and "Life_expectancy" as outcome is ', r2)


$R^2$ for model with "Schooling", "Adult_Mortality" as predictor and "Life_expectancy" as outcome

In [ ]:
### WRITE CODE ###
X = df_train_encoded[['Schooling', 'Adult_Mortality']]
y = df_train_encoded['Life_expectancy']

model = LinearRegression()
model.fit(X,y)

y_pred = model.predict(X)
r2 = r2_score(y,y_pred)
# Print R2 score
print('R^2 for model with "Schooling", "Adult_Mortality" as predictor and "Life_expectancy" as outcome is ', r2)


$R^2$ for model with "Schooling","Adult_Mortality" and "Population" as predictor and "Life_expectancy" as outcome

In [ ]:
### WRITE CODE ###
X = df_train_encoded[['Schooling', 'Adult_Mortality', 'Population']]
y = df_train_encoded['Life_expectancy']

model = LinearRegression()
model.fit(X,y)

y_pred = model.predict(X)
r2 = r2_score(y,y_pred)
# Print R2 score
print('R^2 for model with "Schooling","Adult_Mortality" and "Population" as predictor and "Life_expectancy" as outcome is ', r2)


You can see $R^2$ is always going up as we keep adding features.

This is one drawback of only using $R^2$ to evaluate your model. Adding predictors seems to always improve the predictive ability of your model, though it may not be true.

That is to say, we are not necessarily interested in making a perfect prediciton of our training data. If we were, we would always use all of the predictors available. Rather, we would like to make a perfect prediction of our test data. In this case, adding all the predictors may not be a good idea due to the trade-off between bias and variance. Thus, we are interested in the most predictive features, in the hopes that we can create a simpler model that performs well in the future.

This is why we consider another metric, Adjusted R2.
The adjusted R-squared increases only if the new term improves the model more than would be expected by chance.


$AdjustedR^2$ = $1 - \dfrac{(1-R^2)(n-1)}{(n-p-1)}$<br/>
where:<br/>
n = number of samples<br/>
p = number of features

Fill in the code for calculation of adjusted R2 score

Adjusted $R^2$ for model with "Schooling" as predictor and "Life_expectancy" as outcome

In [ ]:
### WRITE CODE ###
X = df_train_encoded[['Schooling']]
y = df_train_encoded['Life_expectancy']

model = LinearRegression()
model.fit(X,y)

y_pred = model.predict(X)
r2 = r2_score(y,y_pred)

n = len(y)
p = X.shape[1]

adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

# Print Adjusted R2 score
print('Adjusted R^2 for model with "Schooling" as predictor and "Life_expectancy" as outcome is ', adjusted_r2)

Adjusted $R^2$ for model with "Schooling", "Adult_Mortality" as predictor and "Life_expectancy" as outcome.

In [ ]:
### WRITE CODE ###
X = df_train_encoded[['Schooling', 'Adult_Mortality']]
y = df_train_encoded['Life_expectancy']

model = LinearRegression()
model.fit(X,y)

y_pred = model.predict(X)
r2 = r2_score(y,y_pred)

n = len(y)
p = X.shape[1]

adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

# Print Adjusted R2 score
print('Adjusted R^2 for model with "Schooling", "Adult_Mortality" as predictor and "Life_expectancy" as outcome is ', adjusted_r2)

Adjusted $R^2$ for model with "Schooling","Adult_Mortality" and "Population" as predictor and "Life_expectancy" as outcome

In [ ]:
### WRITE CODE ###
X = df_train_encoded[['Schooling', 'Adult_Mortality', 'Population']]
y = df_train_encoded['Life_expectancy']

model = LinearRegression()
model.fit(X,y)

y_pred = model.predict(X)
r2 = r2_score(y,y_pred)

n = len(y)
p = X.shape[1]

adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

# Print Adjusted R2 score
print('Adjusted R^2 for model with "Schooling","Adult_Mortality" and "Population" as predictor and "Life_expectancy" as outcome is ', adjusted_r2)

### K-fold Cross-Validation

However, adjusted $R^2$ is not enough to help us achieve the best model, a more robust method is k-fold cross-validation.

* Randomly split dataset into K equal-sized subsets, or folds
* Treat each fold as validation set (train on all but K'th fold and test on K'th fold only)

* The overall error is then the mean error over all K models.
* Most common are 5- or 10-fold cross-validation

Please implement a 5-fold cross-validation by yourselves to find the best model in terms of Mean Square Error(MSE)

**Do not use sklearn.model_selection.cross_val_score or other built-in cross-validaiton functions**

In [ ]:
# Design a function to implement 5-fold cross-validation.
# The input: training features X, training target y and # of folds f=5.
# The output: the average of MSE over the 5 folds.

lm = LinearRegression()

def cross_val_mse(X, y, f):
    ### Write your code here ###

    shuffled_indices = X.sample(frac=1).index
    X = X.loc[shuffled_indices].reset_index(drop=True)
    y = y.loc[shuffled_indices].reset_index(drop=True)

    size = len(X)//f
    mse_arr = []

    for i in range(f):
        X_val = X[i*size:((i+1)*size)]
        X_train_L = X[:i*size]
        X_train_R = X[(i+1)*size:]

        y_val = y[i*size:((i+1)*size)]
        y_train_L = y[:i*size]
        y_train_R = y[(i+1)*size:]

        X_train = pd.concat([X_train_L, X_train_R], axis=0)
        y_train = pd.concat([y_train_L, y_train_R], axis=0)

        lm.fit(X_train, y_train)
        pred = lm.predict(X_val)
        mse_arr.append(mean_squared_error(y_val, pred))

    return np.mean(mse_arr)

In [ ]:
# By using your above functions, find the best combination of features, which has the lowest averaged MSE
from itertools import combinations
### Write code here ###

best_comb = None
lowest_mse = 25
base_columns = 25

X_train = df_train_encoded.drop(columns=['Life_expectancy'])
y_train = df_train_encoded['Life_expectancy']

# Brute force:
for i in range(base_columns, len(X_train.columns)+1):
    print('current number of features: ' + str(i))
    for c in combinations(X_train.columns, i):
        mse = cross_val_mse(X_train, y_train, 5)
        if mse < lowest_mse:
            lowest_mse = mse
            best_comb = c

In [ ]:
# Print the best features and the corresponding mse
### WRITE CODE ###

print('The best comb of features:' + str(best_comb))
print('with ' + str(len(best_comb)) + ' features')
print('===')
print('Corresponding MSE: ' + str(lowest_mse))

### Test your model
Now, apply your best model to predict the target values from the test feature set "LifeExpectancy_test_modified.csv". We will grade this part based on your prediction error.

Hint: Please be careful on standardization and one-hot encoding (if you use), the test set should be consistent with the training set in terms of any transformation.

Hint2: You may want to modify the previous steps to make the transformation of the test set consistent with the training set.

In [ ]:
df_test_encoded.head()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from statistics import mean
from sklearn import datasets
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
data_features = df_train.drop("Life_expectancy", axis=1)
data_target = df_train['Life_expectancy'].copy()

In [ ]:
data_categorical_features = data_features[['Year', 'Status']]
data_numerical_features = data_features.drop(['Year', 'Status'], axis=1)

# Numerical pipeline
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

# Specifying the features for each pipeline
numerical_features = list(data_numerical_features.columns)
categorical_features = list(data_categorical_features.columns)

# Creating the full pipeline
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, numerical_features),
    ("cat", OneHotEncoder(), categorical_features),
])


In [ ]:
train = full_pipeline.fit_transform(data_features)
test = full_pipeline.transform(df_test)

model = LinearRegression()

model.fit(train, data_target)
y_train_pred = model.predict(train)

In [ ]:
from sklearn.metrics import mean_squared_error

mse_train = mean_squared_error(data_target, y_train_pred)
print("Mean Squared Error on Training Set:", mse_train)

In [ ]:
transformed_numerical_features = numerical_features

# For categorical features, we need to get the new column names from the one-hot encoder
# We need to access the 'cat' transformer from the ColumnTransformer, then the OneHotEncoder itself
onehot_columns = list(full_pipeline.named_transformers_['cat'].get_feature_names_out(categorical_features))

transformed_feature_names = transformed_numerical_features + onehot_columns

coefficients_df = pd.DataFrame({
    'Feature': transformed_feature_names,
    'Coefficient': model.coef_
})

coefficients_df['Coefficient'] = coefficients_df['Coefficient'].apply(lambda x: f'{x:.4f}')

# Sort the DataFrame by 'Coefficient' in descending order (from highest to lowest)
coefficients_df_sorted = coefficients_df.sort_values(by='Coefficient', ascending=False)

coefficients_df_sorted = coefficients_df_sorted.reset_index(drop=True)

coefficients_df_sorted

In [ ]:
import statsmodels.api as sm
import pandas as pd

# Convert the transformed training data to a DataFrame
train_df = pd.DataFrame(train, columns=transformed_feature_names)

# Add a constant to the DataFrame for the intercept
train_df_with_const = sm.add_constant(train_df, prepend=True)
train_df_with_const = train_df_with_const.rename(columns={0: 'Intercept'})

# Create the OLS model with statsmodels using the DataFrame
ols_model = sm.OLS(data_target, train_df_with_const)

# Fit the model
results = ols_model.fit()

# Print out the statistics
model_summary = results.summary()
print(model_summary)

In [ ]:
from sklearn.feature_selection import RFE

# Initialize RFE with the linear regression model
selector = RFE(model, n_features_to_select=1, step=1)

# Fit RFE
selector = selector.fit(train, data_target)

# These are the features selected by RFE
selected_features = np.array(transformed_feature_names)[selector.support_]

# Get the ranking of the features
ranking = selector.ranking_

# Create a DataFrame to display feature ranking
ranking_df = pd.DataFrame({
    'Feature': transformed_feature_names,
    'Ranking': ranking
})

# Sort the DataFrame based on the ranking
ranking_df_sorted = ranking_df.sort_values(by='Ranking')

# Print the ranking of features
print(ranking_df_sorted)

# If you wish to train the model only on selected features
# Filter the training and test sets for selected features
train_selected = selector.transform(train)
test_selected = selector.transform(test)

# Create a new model using only the selected features
model_selected = LinearRegression()
model_selected.fit(train_selected, data_target)

# Predict and evaluate the model as before
y_train_pred_selected = model_selected.predict(train_selected)
mse_train_selected = mean_squared_error(data_target, y_train_pred_selected)
print("Mean Squared Error on Training Set with selected features:", mse_train_selected)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Dictionary to store the MSE for each feature
brute_force_results = {}

# Loop over each feature in the dataset
for feature in transformed_feature_names:
    # Select only the current feature for training
    feature_index = transformed_feature_names.index(feature)
    X_train_feature = train[:, feature_index].reshape(-1, 1)

    # Train the model using only the current feature
    single_feature_model = LinearRegression()
    single_feature_model.fit(X_train_feature, data_target)

    # Predict on the training data
    y_train_pred_feature = single_feature_model.predict(X_train_feature)

    # Calculate the mean squared error
    mse_feature = mean_squared_error(data_target, y_train_pred_feature)
    brute_force_results[feature] = mse_feature

# Sort the results to find the most predictive features (lowest MSE)
sorted_brute_force_results = sorted(brute_force_results.items(), key=lambda x: x[1])

# Display the sorted results
for feature, mse in sorted_brute_force_results:
    print(f'Feature: {feature}, MSE: {mse}')


In [ ]:
#Selected features with lowest p-value, best RFE scores, and lowest MSE
X = df_train[['Schooling', 'HIV_AIDS', 'Income_composition_of_resources', 'Adult_Mortality']]
y = df_train['Life_expectancy']

#Train the model with selected features
model = LinearRegression()
model.fit(X,y)

#Predict on the test set with selected features
y_pred = model.predict(X)

#Calculate the mean squared error on the test set
mse_test = mean_squared_error(data_target, y_pred)
print("Mean Squared Error on Test Set with selected features:", mse_test)

In [ ]:
#end

### Individual student contribution
Student 1 Joy Lin - Q1-Q6 </br>
Student 2 Yash Garg - Q7-Q9 </br>
Student 3 Binglei Roger Zhang - Goodness of fit </br>
Student 4 Steven Chen - K-fold Cross-Validation </br>
Student 5 Helen Chen - Test Your Model
